In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_4265/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/20 00:04:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/20 00:04:47 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for Friends table
friends_data = {
    'user1': [2, 1, 4, 1, 1, 2, 7, 8, 3],
    'user2': [1, 3, 1, 5, 6, 6, 2, 3, 9]
}

# Create DataFrame for Friends
friends_df = pd.DataFrame(friends_data)

# Display the DataFrame
print("Friends table:")
print(friends_df)



df_person = spark.createDataFrame(friends_df)
df_person.createOrReplaceTempView("Friends")



Friends table:
   user1  user2
0      2      1
1      1      3
2      4      1
3      1      5
4      1      6
5      2      6
6      7      2
7      8      3
8      3      9


In [19]:
query = """
    with all_user as (
        select
            distinct user_id, friend_id
        from (
            select user1 user_id, user2 friend_id from Friends 
            union all
            select user2 user_id, user1 friend_id from Friends 
        )
    )
    
    select
        distinct
        user_id,
        round(count(*) over(partition by user_id) / (select count(distinct user_id) from all_user) * 100, 2) as percentage_popularity 
    from all_user

"""

In [20]:
spark.sql(query).show()

+-------+---------------------+
|user_id|percentage_popularity|
+-------+---------------------+
|      1|                55.56|
|      2|                33.33|
|      3|                33.33|
|      4|                11.11|
|      5|                11.11|
|      6|                22.22|
|      7|                11.11|
|      8|                11.11|
|      9|                11.11|
+-------+---------------------+

